In [1]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

import pandas as pd

In [2]:
# Montana's "Index-of-Terms" sheet is in XLSX format: 
#   https://docs.google.com/spreadsheets/d/1R5w5tcIGm9dDRhC4H6NmhcHV7J5zx7FQ/edit#gid=1133203354
# I copied and converted to Google Sheet format in MAM@lbl.gov google drive
#   https://docs.google.com/spreadsheets/d/1lj4OuEE4IYwy2v7RzcG79lHjNdFwmDETMDTDaRAWojY/edit#gid=1133203354

# there's also a "Controlled Terms" tab

In [3]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
IOT_SPREADSHEET_ID = '1lj4OuEE4IYwy2v7RzcG79lHjNdFwmDETMDTDaRAWojY'
IOT_RANGE_NAME = 'Glossary of terms!A1:Z'

# resuing google_api_credentials.json from https://github.com/cancerDHC/sheet2linkml
client_secret_flie = '../../sheet2linkml/google_api_credentials.json'


In [4]:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            client_secret_flie, SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=25191608779-77qsof3vglumvlbn2d9n35p55ntncb8e.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49539%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=A4wA3Y8gEsYZPq2OiqAv7yEKrzaf4l&access_type=offline


In [5]:
service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()

In [6]:
result = sheet.values().get(spreadsheetId=IOT_SPREADSHEET_ID,
                            range=IOT_RANGE_NAME).execute()

In [7]:
iot_glossary = pd.DataFrame(result['values'],columns=result['values'][0]).drop(0)

In [8]:
iot_glossary

,Column Header,name,Definition,Guidance,Expected Value,syntax,Category,Associated Packages,Origin,Notes,GitHub Ticket
1,Unique ID,unique_ID,A globally unique identifier assigned to the b...,Field REQUIRED for ALL sample submission. Opti...,,{text},sample identification,all,,v6 has unique ID and sample name synonymous. B...,None
2,Sample Name,samp_name,A local identifier or name that for the materi...,ID that is present on the shipped sample. Huma...,,{text},sample identification,all,MIxS,v6,None
3,Analysis/Data Type,investigation_type,,This field is constrained to contain only a se...,,drop down selection list,sample identification,all,MIxS,"enumeration values are EMSL origin, MIxS has d...",None
4,Sample Type,package,,MIxS Package,,{text},sample identification,all,MIxS,"This is assumed by the package, but I think sh...",None
5,Source Material ID,source_mat_ID,A unique identifier assigned to a material sam...,A unique identifier assigned to an original ma...,,{text},sample identification,all,MIxS,New v6 definition is better to indicate bio sa...,None
...,...,...,...,...,...,...,...,...,...,...,...
244,wastewater type,wastewater_type,"the origin of wastewater such as human waste, ...",,,{text},,water_extract_biosolid,MIxS,None,None
245,water content,water_content,Water content measurement,,,{float},,soil; sediment; water_extract_soil,MIxS,None,None
246,water content method,water_content_meth,Reference or method used in determining the wa...,,,{PMID}|{DOI}|{URL},,soil; sediment,MIxS,None,None
247,water current,water_current,Measurement of magnitude and direction of flow...,,,{float} {unit},,pore_water; water,MIxS,None,None
